In [67]:
import numpy as np
from astropy import units as u 
from astropy.io import fits
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy.table import QTable
from reproject import reproject_interp
from astropy import stats
import os 
from astropy.table import QTable, join, vstack, hstack, Column


plt.style.use('paper.mplstyle')

In [68]:
# Define names and filenames...
root_dir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/'

In [69]:
def get_files(endswith, root_dir): 

    file_list = []
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(endswith):
                if '628e' in file: 
                    continue
                else: 
                    print(os.path.join(root, file))
                    file_list += [os.path.join(root, file)]

    print('TOTAL NUMBER OF FILES:', len(file_list))
    return(file_list)

files_maps = get_files('_hst_ha_err.fits', root_dir)

/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc3627/hst_contsub/ngc3627_hst_ha_err.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc4254/hst_contsub/ngc4254_hst_ha_err.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1512/hst_contsub/ngc1512_hst_ha_err.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1433/hst_contsub/ngc1433_hst_ha_err.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc5068/hst_contsub/ngc5068_hst_ha_err.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc4321/hst_contsub/ngc4321_hst_ha_err.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1365n/hst_contsub/ngc1365n_hst_ha_err.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc4303/hst_contsub/ngc4303_hst_ha_err.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1087/hst_contsub/ngc1087_hst_ha_err.fits
/Users/abarnes/Dropbox/work/Smallprojects/ga

In [70]:
output_stats = []

for file_maps in files_maps: 

    # Loading files...
    hstha_hdu = fits.open(file_maps)[0]
    gal = file_maps.split(root_dir)[-1].split('/')[0]

    # Get RMS for whole map... 
    std = np.nanstd(hstha_hdu.data)
    mad_std = stats.mad_std(hstha_hdu.data, ignore_nan=True)
    mean = np.nanmean(hstha_hdu.data)
    median = np.nanmedian(hstha_hdu.data)

    pix_scale = (np.absolute(hstha_hdu.header['CD1_1'])*u.deg).to('arcsec')
    pix_area = pix_scale**2
    median_arcsec2 = (median/pix_area.value)/1e3 # erg/s/cm2/arcsec2 10-17

    output_stats += [[std, mean, median, median_arcsec2]]
    print("Galaxy:", gal, "STD:", std, "MAD_STD:", mad_std, "MEAN:", mean, "MEDIAN:", median, "MEDIAN_ARCSEC2:", median_arcsec2)

Galaxy: ngc3627 STD: 920.2913 MAD_STD: 13.117781011970113 MEAN: 157.14175 MEDIAN: 147.61073 MEDIAN_ARCSEC2: 94.03488429851474
Galaxy: ngc4254 STD: 973.1109 MAD_STD: 11.907216935516663 MEAN: 128.5448 MEDIAN: 118.87257 MEDIAN_ARCSEC2: 75.72733893693552
Galaxy: ngc1512 STD: 1342.9034 MAD_STD: 9.554262332803042 MEAN: 140.4964 MEDIAN: 125.729454 MEDIAN_ARCSEC2: 80.09549455346998
Galaxy: ngc1433 STD: 2059.9712 MAD_STD: 19.41782241846571 MEAN: 232.75398 MEDIAN: 206.28331 MEDIAN_ARCSEC2: 131.4120374861947
Galaxy: ngc5068 STD: 937.4707 MAD_STD: 9.263956348779404 MEAN: 117.493645 MEDIAN: 111.91892 MEDIAN_ARCSEC2: 71.29754527191031
Galaxy: ngc4321 STD: 1195.2693 MAD_STD: 12.984352241756092 MEAN: 167.12244 MEDIAN: 154.03271 MEDIAN_ARCSEC2: 98.12598461492553
Galaxy: ngc1365n STD: 1703.2129 MAD_STD: 15.303859161058146 MEAN: 178.81073 MEDIAN: 158.79604 MEDIAN_ARCSEC2: 101.16044100340002
Galaxy: ngc4303 STD: 1274.2167 MAD_STD: 10.00041619712583 MEAN: 130.88756 MEDIAN: 118.13002 MEDIAN_ARCSEC2: 75.2543

In [71]:
filters = [
            ['IC~5332', 'F658N WFC3'],
            ['NGC~0628', 'F658N ACS'],
            ['NGC~1087', 'F657N WFC3'],
            ['NGC~1300', 'F658N ACS'],
            ['NGC~1300', 'F658N ACS'],
            ['NGC~1365', 'F657N WFC3'],
            ['NGC~1385', 'F657N WFC3'],
            ['NGC~1433', 'F657N WFC3'],
            ['NGC~1512', 'F658N WFC3'],
            ['NGC~1566', 'F658N WFC3'],
            ['NGC~1672', 'F658N ACS'],
            ['NGC~2835', 'F658N WFC3'],
            ['NGC~3351', 'F658N WFC3'],
            ['NGC~3627', 'F658N WFC3'],
            ['NGC~4254', 'F657N WFC3'],
            ['NGC~4303', 'F658N WFC3'],
            ['NGC~4321', 'F657N WFC3'],
            ['NGC~5068', 'F658N WFC3'],
            ['NGC~7496', 'F657N WFC3'],
            ]

filters = np.array(filters)

table_filters = QTable(filters, names=('gal_name', 'filter'))
table_filters

gal_name,filter
str10,str10
IC~5332,F658N WFC3
NGC~0628,F658N ACS
NGC~1087,F657N WFC3
NGC~1300,F658N ACS
NGC~1300,F658N ACS
NGC~1365,F657N WFC3
NGC~1385,F657N WFC3
NGC~1433,F657N WFC3
NGC~1512,F658N WFC3


In [72]:
# Get into a astropy table the output of the stats of the maps to .tex
gals = []
for file_maps in files_maps:
    gal = file_maps.split(root_dir)[-1].split('/')[0]
    gals += [gal]

output_stats = np.array([output_stats][0], dtype=np.float64)
gals = np.array(gals)

for i, gal in enumerate(gals):
    gal = gal.upper()
    if 'NGC' in gal:
        gal_split = gal.split('NGC')
        gal = 'NGC~' + gal_split[1]
    if 'IC' in gal:
        gal_split = gal.split('IC')
        gal = 'IC~' + gal_split[1]
    if '628C' in gal:
        gal_split = gal.split('628C')
        gal = gal_split[0] + '0628'
    gals[i] = gal

table1 = Column(gals, name='gal_name')
table2 = QTable(output_stats, names=('std', 'mean', 'median', 'median_arcsec2'))

table = hstack([table1, table2])
table.sort('gal_name')
table

gal_name,std,mean,median,median_arcsec2
str8,float64,float64,float64,float64
IC~5332,1404.157958984375,147.56942749023438,133.87310791015625,85.2833798357043
NGC~0628,349.0311279296875,95.73289489746094,89.04512023925781,56.72587221159494
NGC~1087,1853.5106201171875,167.70982360839844,145.26837158203125,92.54269139743124
NGC~1300,216.422119140625,62.72904586791992,59.69546127319336,38.02877798015994
NGC~1365,1703.212890625,178.81072998046875,158.79603576660156,101.16044100340002
NGC~1385,1576.65771484375,176.8993682861328,159.90713500976562,101.86826276289422
NGC~1433,2059.97119140625,232.7539825439453,206.28330993652344,131.4120374861947
NGC~1512,1342.9034423828125,140.49639892578125,125.72945404052734,80.09549455346998
NGC~1566,1304.150634765625,134.142822265625,119.60688018798828,76.19513099584896


In [73]:
table = join(table_filters, table, keys='gal_name')
table

gal_name,filter,std,mean,median,median_arcsec2
str10,str10,float64,float64,float64,float64
IC~5332,F658N WFC3,1404.157958984375,147.56942749023438,133.87310791015625,85.2833798357043
NGC~0628,F658N ACS,349.0311279296875,95.73289489746094,89.04512023925781,56.72587221159494
NGC~1087,F657N WFC3,1853.5106201171875,167.70982360839844,145.26837158203125,92.54269139743124
NGC~1300,F658N ACS,216.422119140625,62.72904586791992,59.69546127319336,38.02877798015994
NGC~1300,F658N ACS,216.422119140625,62.72904586791992,59.69546127319336,38.02877798015994
NGC~1365,F657N WFC3,1703.212890625,178.81072998046875,158.79603576660156,101.16044100340002
NGC~1385,F657N WFC3,1576.65771484375,176.8993682861328,159.90713500976562,101.86826276289422
NGC~1433,F657N WFC3,2059.97119140625,232.7539825439453,206.28330993652344,131.4120374861947
NGC~1512,F658N WFC3,1342.9034423828125,140.49639892578125,125.72945404052734,80.09549455346998


In [79]:
table_out = table['gal_name', 'filter', 'median_arcsec2']

formats={
        'gal_name': '%s',
        'filter': '%s',
        'median_arcsec2': '%0.1f'}

table_out.write('./tabs/galmap_errprops.tex', formats=formats, overwrite=True)

In [76]:
# Reorganise to match names in .fits files
gals = table['gal_name']

# for i, gal in enumerate(gals):
#     if '~' in gal:
#         gal_split = gal.split('~')
#         gal = gal_split[0] + gal_split[1]
#     table['gal_name'][i] = gal

table.write('./tabs/galmap_props.fits', overwrite=True)
table

gal_name,filter,std,mean,median,median_arcsec2
str10,str10,float64,float64,float64,float64
IC~5332,F658N WFC3,1404.157958984375,147.56942749023438,133.87310791015625,85.2833798357043
NGC~0628,F658N ACS,349.0311279296875,95.73289489746094,89.04512023925781,56.72587221159494
NGC~1087,F657N WFC3,1853.5106201171875,167.70982360839844,145.26837158203125,92.54269139743124
NGC~1300,F658N ACS,216.422119140625,62.72904586791992,59.69546127319336,38.02877798015994
NGC~1300,F658N ACS,216.422119140625,62.72904586791992,59.69546127319336,38.02877798015994
NGC~1365,F657N WFC3,1703.212890625,178.81072998046875,158.79603576660156,101.16044100340002
NGC~1385,F657N WFC3,1576.65771484375,176.8993682861328,159.90713500976562,101.86826276289422
NGC~1433,F657N WFC3,2059.97119140625,232.7539825439453,206.28330993652344,131.4120374861947
NGC~1512,F658N WFC3,1342.9034423828125,140.49639892578125,125.72945404052734,80.09549455346998
